In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

In [3]:
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)

In [4]:
def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

In [5]:
# lgb 
# 0.7917	0.30304	0.88216	0.89852	0.96327	0.89229	0.66823	0.91991	0.80477
with open('./sub/CVPR_2022_lgb_score.json', 'r') as f:
    test_data_7917 = json.load(f)
test_data_7917 = get_df(test_data_7917)

# pp super lr
# 0.79251	0.30632	0.88203	0.89841	0.96363	0.89552	0.66735	0.92054	0.80633
with open('./sub/CVPR_2022_paddle_superliner_score.json', 'r') as f:
    test_data_79251 = json.load(f)
test_data_79251 = get_df(test_data_79251)

# nn MLP+lstm2y+MLP 加权层，bc400,001lr
# 0.79253	0.29919	0.8823	0.89703	0.96335	0.89861	0.6753	0.92263	0.80182
with open('./sub/CVPR_2022_torch_ohe_2lstm_4logits_weight_kednall_tanh1.json', 'r') as f:
    test_data_79253 = json.load(f)
test_data_79253 = get_df(test_data_79253)

# nn MLP+transformer decoder encoder
# 0.79282	0.32	0.87896	0.89657	0.96108	0.89295	0.66939	0.92099	0.8026
with open('./sub/CVPR_2022_transformer_encoder_decoder_tanh.json', 'r') as f:
    test_data_79282 = json.load(f)
test_data_79282 = get_df(test_data_79282)

# nn lstm2y_bc400_lr001_20220512.json
# 0.79351	0.30797	0.88162	0.89854	0.96421	0.89542	0.67249	0.92281	0.80505
with open('./sub/CVPR_2022_lstm2y_catall_tanh.json', 'r') as f:
    test_data_79351 = json.load(f)
test_data_79351 = get_df(test_data_79351)

# nn 7937 target 8 best tanh
# 0.79353	0.29968	0.88367	0.89992	0.96528	0.89853	0.67058	0.92237	0.80825
with open('./sub/CVPR_2022_lstm2y_catall_tanh_sig.json', 'r') as f:
    test_data_79353 = json.load(f)
test_data_79353 = get_df(test_data_79353)

#nn 79066 8target 同步建模，取sig best
# 0.79395	0.29616	0.88538	0.90162	0.96621	0.9011	0.67295	0.92239	0.80579	已完成	2022/5/14 16:45
with open('./sub/CVPR_2022_lstm2y_catall_pair_sig.json', 'r') as f:
    test_data_79395 = json.load(f)
test_data_79395 = get_df(test_data_79395)

# nn 79395 取tanh1
# 0.79414	0.31042	0.88253	0.89818	0.96561	0.89796	0.67172	0.92211	0.80456	已完成	2022/5/14 19:51
with open('./sub/CVPR_2022_lstm2y_catall_tanh1_sig.json', 'r') as f:
    test_data_79414 = json.load(f)
test_data_79414 = get_df(test_data_79414)

# nn 79414 ohe step 37*93, lstm2y稠密_bc400_lr001_tanh1_ohe_20220515
# 0.79455	0.31011	0.8841	0.89642	0.96365	0.90124	0.67184	0.92352	0.80555	已完成	2022/5/15 16:49
with open('./sub/CVPR_2022_ohe_lstm2y_catall_tanh1_sig.json', 'r') as f:
    test_data_79455 = json.load(f)
test_data_79455 = get_df(test_data_79455)

In [6]:
out = pd.merge(test_data_7917[['id']+target_cols].rename(columns={x:x+'_7917' for x in target_cols}),
               test_data_79251[['id']+target_cols].rename(columns={x:x+'_79251' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79253[['id']+target_cols].rename(columns={x:x+'_79253' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79282[['id']+target_cols].rename(columns={x:x+'_79282' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79351[['id']+target_cols].rename(columns={x:x+'_79351' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79353[['id']+target_cols].rename(columns={x:x+'_79353' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79395[['id']+target_cols].rename(columns={x:x+'_79395' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79414[['id']+target_cols].rename(columns={x:x+'_79414' for x in target_cols}),
               on='id')
out = pd.merge(out,
               test_data_79455[['id']+target_cols].rename(columns={x:x+'_79455' for x in target_cols}),
               on='id')

In [7]:
for y_ in target_cols:
    if y_ == target_cols[0]:
        out[y_] = (out[f'{y_}_79282']).rank().astype(int)-1
    elif y_ == target_cols[1]:
        out[y_] = (out[f'{y_}_79395']*0.8+out[f'{y_}_79455']*0.05 + out[f'{y_}_79353']*0.05 + out[f'{y_}_79414']*0.05 + out[f'{y_}_7917']*0.05).rank().astype(int)-1
    elif y_ == target_cols[2]:
        out[y_] = (out[f'{y_}_79395']*1).rank().astype(int)-1
    elif y_ == target_cols[3]:
        out[y_] = (out[f'{y_}_79395']*0.8+out[f'{y_}_79353']*0.05 + out[f'{y_}_79414']*0.05 + out[f'{y_}_79251']*0.05 + out[f'{y_}_79351']*0.05).rank().astype(int)-1
    elif y_ == target_cols[4]:
        out[y_] = (out[f'{y_}_79455']*0.45+out[f'{y_}_79353']*0.02 + out[f'{y_}_79414']*0.02 + out[f'{y_}_79395']*0.45 + out[f'{y_}_79253']*0.02).rank().astype(int)-1
    elif y_ == target_cols[5]:
        out[y_] = (out[f'{y_}_79253']*1).rank().astype(int)-1
    elif y_ == target_cols[6]:
        out[y_] = (out[f'{y_}_79351']*0.1+out[f'{y_}_79455']*0.6 + out[f'{y_}_79353']*0.1 + out[f'{y_}_79414']*0.1 + out[f'{y_}_79253']*0.1).rank().astype(int)-1
    elif y_ == target_cols[7]:
        out[y_] = (out[f'{y_}_79455']*0.05+out[f'{y_}_79353']*0.7 + out[f'{y_}_79395']*0.05 + out[f'{y_}_79251']*0.2).rank().astype(int)-1
    else:
        print('error')

In [8]:
def to_sub(test_df, test_data, name='CVPR_2022_lgb_base_log1p_grid'):
    for i in tqdm(test_df[['id']+target_cols].values):
        id_ = i[0]
        for k,v in enumerate(target_cols):
            k += 1
            test_data[id_][v] = i[k]
            
    with open(f'./sub/{name}.json', 'w') as f:
        json.dump(test_data, f)

#第一个提交，对应a榜 0.79806，第一个提交对较强的模型有很高的权重
to_sub(out, test_data, name='sub0')

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99500 [00:00<?, ?it/s]

In [9]:
for y_ in target_cols:
    if y_ == target_cols[0]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.03+\
                   out[f'{y_}_79282']*0.4+out[f'{y_}_79351']*0.1+out[f'{y_}_79353']*0.03+\
                   out[f'{y_}_79395']*0.03+out[f'{y_}_79414']*0.1+out[f'{y_}_79455']*0.1).rank().astype(int)-1
    elif y_ == target_cols[1]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.4+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.08).rank().astype(int)-1
    elif y_ == target_cols[2]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.4+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.08).rank().astype(int)-1
    elif y_ == target_cols[3]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.4+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.08).rank().astype(int)-1
    elif y_ == target_cols[4]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.08+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.4).rank().astype(int)-1
        
    elif y_ == target_cols[5]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.4+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.08+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.08).rank().astype(int)-1
    elif y_ == target_cols[6]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.08+\
                   out[f'{y_}_79395']*0.08+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.4).rank().astype(int)-1
    elif y_ == target_cols[7]:
        out[y_] = (out[f'{y_}_7917']*0.1+out[f'{y_}_79251']*0.1+out[f'{y_}_79253']*0.08+\
                   out[f'{y_}_79282']*0+out[f'{y_}_79351']*0.08+out[f'{y_}_79353']*0.4+\
                   out[f'{y_}_79395']*0.08+out[f'{y_}_79414']*0.08+out[f'{y_}_79455']*0.08).rank().astype(int)-1
    else:
        print('error')

In [10]:
# 第二个提交，对应a榜 0.79727，第二个提交，强模型，打分模型，nn编码模型分别给4：2：4的权重
to_sub(out, test_data, name='sub1')

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99500 [00:00<?, ?it/s]